# Addestramento modelli

In [ ]:
import warnings
import numpy as np
import pandas as pd
import os
import sys  
from datetime import datetime

In [ ]:
warnings.filterwarnings("ignore")
sys.path.insert(0, "..")

In [ ]:
import utility

In [ ]:
from keras.models import load_model

In [ ]:
model_trained_base_path = "TrainingDirectory"
plot_base_path = os.path.join(model_trained_base_path, "Risultati")
dataset_path = os.path.join("..", "Preprocessing", "Dataset", "dataset_training.csv")

### Import dei dati

In [ ]:
df = utility.import_dataset(dataset_path, "2021", "2023")
display(df)

### OneHot encoding e normalizzazione

In [ ]:
df, scaler = utility.processing_dataset(df)

### Splitting in training, validation e test

In [ ]:
df_train, df_validation, df_test = utility.split_dataset(df, "2021-03", "2022-04")

### Prediction models

##### Model 1 - PAPER0 - LSTMwithoutOtherFeature

In [ ]:
base_path = os.path.join(model_trained_base_path, "model1")
model = load_model(os.path.join(base_path, "weights-model.h5"))

In [ ]:
time_steps = 24 * 7
dim_other_feature = 24 + 7 + 12 + 1

In [ ]:
test = utility.windowed_dataset(df_test, time_steps)
y_pred_test = model.predict(utility.get_x(test, "CaricoTotale")[0])
y_pred_test_inv = scaler.inverse_transform(y_pred_test)
df1 = utility.add_prediction(df_test, y_pred_test_inv, time_steps, "Predizione Modello 1")
df1 = df1[["Predizione Modello 1"]]

##### Model 2 - PAPER12 - LSTMwithOtherFeature

In [ ]:
base_path = os.path.join(model_trained_base_path, "model2")
model = load_model(os.path.join(base_path, "weights-model.h5"))

In [ ]:
time_steps = 24 * 3
dim_other_feature = 24 + 7 + 12 + 1

In [ ]:
test = utility.windowed_dataset(df_test, time_steps)
y_pred_test = model.predict(utility.get_x(test, "CaricoTotale"))
y_pred_test_inv = scaler.inverse_transform(y_pred_test)
df2 = utility.add_prediction(df_test, y_pred_test_inv, time_steps, "Predizione Modello 2")
df2 = df2[["Predizione Modello 2"]]

##### Model 3 - PAPER13 - Conv1D + LSTM

In [ ]:
base_path = os.path.join(model_trained_base_path, "model3")
model = load_model(os.path.join(base_path, "weights-model.h5"))

In [ ]:
time_steps = 24 * 3
dim_other_feature = 24 + 7 + 12 + 1

In [ ]:
test = utility.windowed_dataset(df_test, time_steps)
y_pred_test = model.predict(utility.get_x(test, "CaricoTotale"))
y_pred_test_inv = scaler.inverse_transform(y_pred_test)
df3 = utility.add_prediction(df_test, y_pred_test_inv, time_steps, "Predizione Modello 3")
df3 = df3[["Predizione Modello 3"]]

##### Model 4 - PAPER18 - Wavenet

In [ ]:
base_path = os.path.join(model_trained_base_path, "model4")
model = load_model(os.path.join(base_path, "weights-model.h5"))

In [ ]:
time_steps = 24 * 5
dim_other_feature = 24 + 7 + 12 + 1

In [ ]:
test = utility.windowed_dataset(df_test, time_steps)
y_pred_test = model.predict(utility.get_x(test, "CaricoTotale"))
y_pred_test_inv = scaler.inverse_transform(y_pred_test)
df4 = utility.add_prediction(df_test, y_pred_test_inv, time_steps, "Predizione Modello 4")
df4 = df4[["Predizione Modello 4"]]

### Plot curve di predizione

In [ ]:
df = df_test[["Carico totale"]]
df = df.merge(df1, left_index=True, right_index=True)
df = df.merge(df2, left_index=True, right_index=True)
df = df.merge(df3, left_index=True, right_index=True)
df = df.merge(df4, left_index=True, right_index=True)

In [ ]:
counter = 0
ft = "%Y-%m-%d %H:%M:%S"
start = datetime.strptime("2022-04-08 00:00:00", ft)
end = datetime.strptime("2022-04-14 23:00:00", ft)

title = "Confronto curve di predizione dei quattro modelli (Aprile 2022)"
xlabel = None #"Ora"
ylabel = "Carico elettrico (W)"
dim = (15,7)
marker = False
fields = ["Carico totale", "Predizione Modello 1", "Predizione Modello 2", "Predizione Modello 3", "Predizione Modello 4"]
colors = ["#121212", "#FF7518", "#FFBF00", "#6495ED", "#93C572"]
lw = 3
utility.plot_distribution(df, fields, start, end, dim, title, xlabel, ylabel, marker=marker, colors=colors, linewidth=lw, base_path=plot_base_path)

In [ ]:
counter = 0
ft = "%Y-%m-%d %H:%M:%S"
start = datetime.strptime("2022-05-11 00:00:00", ft)
end = datetime.strptime("2022-05-11 23:00:00", ft)

title = None #"Confronto curva di carico e predizione da " + str(start) + " a " + str(end)
xlabel = None #"Ora"
ylabel = "Carico elettrico (W)"
dim = (15,7)
fields = ["Carico totale", "Predizione Modello 1", "Predizione Modello 2", "Predizione Modello 3", "Predizione Modello 4"]
colors = ["#121212", "#FF7F50", "#FFBF00", "#6495ED", "#93C572"]
markers = [".-", "o--", "s--", "*--", "D--"]
ms = 7
lw = 3
utility.plot_multiple_distribution(df, fields, start, end, dim, title, xlabel, ylabel, colors, markers, ms, lw, base_path=plot_base_path)

### Plot errori settimana

In [ ]:
df_e1 = utility.get_df_error_days_of_week(df, "Carico totale", "Predizione Modello 1")
df_e2 = utility.get_df_error_days_of_week(df, "Carico totale", "Predizione Modello 2")
df_e3 = utility.get_df_error_days_of_week(df, "Carico totale", "Predizione Modello 3")
df_e4 = utility.get_df_error_days_of_week(df, "Carico totale", "Predizione Modello 4")

In [ ]:
values_df1 = np.round(df_e1["mape"], 2)
values_df2 = np.round(df_e2["mape"], 2)
values_df3 = np.round(df_e3["mape"], 2)
values_df4 = np.round(df_e4["mape"], 2)
values = [values_df1, values_df2, values_df3, values_df4]

keys = ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato", "Domenica"]
labels = ["Modello 1", "Modello 2", "Modello 3", "Modello 4"]
colors = ["#FF7F50", "#FFBF00", "#6495ED", "#93C572"]

bar_width = 0.20
dim = (20,8)

title = None #"MAPE (%) sul test set dei modelli addestrati sui giorni della settimana"
ylabel = "MAPE (%)"
xlabel = None #"Giorni della settimana"

utility.multiple_bar_plot(keys, values, labels, bar_width, xlabel, ylabel, title, colors, dim, base_path=plot_base_path)

In [ ]:
values_df1 = np.round(df_e1["mae"], 0)
values_df2 = np.round(df_e2["mae"], 0)
values_df3 = np.round(df_e3["mae"], 0)
values_df4 = np.round(df_e4["mae"], 0)
values = [values_df1, values_df2, values_df3, values_df4]

keys = ["Lunedì", "Martedì", "Mercoledì", "Giovedì", "Venerdì", "Sabato", "Domenica"]
labels = ["Modello 1", "Modello 2", "Modello 3", "Modello 4"]
colors = ["#FF7F50", "#FFBF00", "#6495ED", "#93C572"]

bar_width = 0.20
dim = (20,8)

title = None #"MAE sul test set dei modelli addestrati sui giorni della settimana"
ylabel = "MAE (W)"
xlabel = None #"Giorni della settimana"

utility.multiple_bar_plot(keys, values, labels, bar_width, xlabel, ylabel, title, colors, dim, base_path=plot_base_path)

### Plot errori giornate

In [ ]:
df_e1 = utility.get_df_errors(df, "Carico totale", "Predizione Modello 1")
df_e2 = utility.get_df_errors(df, "Carico totale", "Predizione Modello 2")
df_e3 = utility.get_df_errors(df, "Carico totale", "Predizione Modello 3")
df_e4 = utility.get_df_errors(df, "Carico totale", "Predizione Modello 4")

In [ ]:
values_df1 = np.round(df_e1["mae"], 2)
values_df2 = np.round(df_e2["mae"], 2)
values_df3 = np.round(df_e3["mae"], 2)
values_df4 = np.round(df_e4["mae"], 2)
dff = pd.DataFrame({
    "Modello 1": values_df1,
    "Modello 2": values_df2,
    "Modello 3": values_df3,
    "Modello 4": values_df4
})

In [ ]:
dff["DoW"] = dff.index.weekday

In [ ]:
dff = dff[dff["DoW"] <= 4]

In [ ]:
dff.sort_values(by="Modello 2")

In [ ]:
dfff = dff < 3
dfff.sum()

In [ ]:
counter = 0
ft = "%Y-%m-%d %H:%M:%S"
start = dff.index.min()
end = dff.index.max()

title = "Confronto curva di carico e predizione da " + str(start) + " a " + str(end)
xlabel = "Ora"
ylabel = "W"
dim = (20,7)
marker = True
fields = ["Modello 1", "Modello 2", "Modello 3", "Modello 4"]
colors = ["#9966CC", "#FDBA21", "#6495ED", "#93C572"]
markers = ["o-", "s-", "D-", "d-"]
ms = 7
utility.plot_multiple_distribution(dff, fields, start, end, dim, title, xlabel, ylabel, colors, markers, ms, base_path=None)